# Curs 7: Preprocesarea datelor si invatare nesupervizata

Invatarea nesupervizata a datelor trateaza cazul in care datele nu sunt etichetate sau nu au vreo alta indicatie - fie ea de natura continua sau discreta - asociata. Orice problema de clasificare sau de regresie se poate transforma intr-o problema de invatare de tip nesupervizat, prin inlaturarea etichetei aferente fiecarei inregistrari. 

Discutam in acest curs doua tipuri de invatare nesupervizata: 
* transformare nesupervizata a datelor
* clustering

Aceste operatii se folosesc frecvent in etapa de explorare a datelor, de exemplu pentru a capata rapid o idee despre structura datelor. In alte cazuri se aplica pe post de metode de preprocesare, de exemplu pentru a aduce valorile de pe dimensiuni diferite la aceleasi scale sau pentru a micsora numarul de date. 

## 7.1 Transformarea nesupervizata a datelor

Transformarea nesupervizata a datelor vizeaza obtinerea unei noi reprezentari a setului initial cu scopul de a le face mai suor de inteles de oameni sau mai utile pentru un algoritm de ML. De exemplu, reducerea de la un numar mare de dimensiuni la 2 sau 3 dimensiuni permite reprezentarea grafica si obtinerea rapida a unei vederi initiale bune asupra datelor. 

### 7.1.1. Scalarea datelor

Anumiti algoritmi, precum cei bazati pe calcul de distante sau cei ce lucreaza cu stochastic gradient descent sunt senzitivi la scala datelor: ei prefera ca datele sa fie cu acelasi odin de marime. De exemplu, pentru cazul in care pentru doi vectori $n$-dimensionali $\mathbf{x} = (x_1, \dots, x_n)$ respectiv $\mathbf{y} = (y_1, \dots, y_n)$ se calculeaza distanta dintre ei cu metrica Euclidiana:
$$
d(\mathbf{x}, \mathbf{y}) = \sqrt{\sum\limits_{i=1}^n (x_i - y_i)^2 }
$$
daca pentru primul indice (prima dimensiune) valorile sunt de ordinul sutelor iar pentru restul dimensiunilor valorile sunt de ordinul zecilor de unitati, atunci valoarea distantei este practic determinata doar de diferenta intre prima dimensiune a fiecarui vector; celelalte dimensiuni nu au nicio influenta.  

Exista urmatoarele metode populare de scalare:
1. scalarea min-max: toate trasaturile (dimensiunile) sunt transformate in mod independent, astfel incat valorile minime si maxime pe respectiva trasatura sa fie intre un minim si un maxim date. Implementarea e simpla, se calculeaza pentru fiecare dimensiune minimul si maximul, apoi diferenta dintre fiecare valoare si minimul seriei sale este imartita la diferenta intre maximul si minimul seriei din care face parte:
1. standardizarea: fiecare dimensiune e astfel transformata incat sa aiba media zero si deviatia standard 1; aceasta se obtine prin: se calculeaza media si deviatia standard pentru fiecare dimensiunne; fiecare serie (dimensiune) se transforma prin impartirea diferentei dintre valorile din seria originara si media seriei la deviatia standard; 
1. scalarea robusta: ca la punctul anterior, dar se folosesc mediana si quartile ale datelor din fiecare serie, independent;
1. normalizarea: se imparte orice vector (presupus nenul) la norma sa. Norma se algee convenabil. In urma transformarii, orice vector va avea norma 1 si se va gasi pe hipersfera de raza 1 centarta in origine. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
data_cancer = ...
data_names = ...
print(data_names)

In [ ]:
#creare dataframe
df_cancer = ...
df_cancer.head()

In [ ]:
#descriere succinta
...

Se observa discrepantele majore intre valorile minime si maxime:

In [ ]:
df_cancer.describe().loc[['min', 'max']]

Reprezentare grafica:

In [ ]:
feature1 = 'mean concavity'
feature2 = 'worst perimeter'
plt.figure(figsize=(20, 10))
plt.axis('equal')
plt.xlabel(feature1)
plt.ylabel(feature2)
plt.scatter(df_cancer[feature1], df_cancer[feature2])

In [ ]:
# import seaborn as sns
# sns.set(style="ticks")
# sns.pairplot(df_cancer)

In cazul in care setul de date esta impartit in set de antrenare si set de validare sau de testare, parametrii folositi pentru transformarea datelor trebuie sa fie retinuti si reutilizati pentru a face aceleasi transformari pe setul exterior celui de antrenare. Este gresit ca seturile de testare sau de validare sa fie transformate cu alte valori, pentru ca modelul (de clasificare/regresie/clustering) determinat pe setul de antrenare are sanse reale sa nu functioneze deloc bine.

Exemple de aplicare a transformarilor:

In [ ]:
#etichetele y_* sunt utile pentru a demonstra utilitatea scalarii
X_train, X_test, y_train, y_test = ...

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
min_max_scaler = ...
#se observa ca datele din X_train nu sunt modificare
...

In [ ]:
#dar obiectul de scalare castiga in starea lui valorile minime si maxime pe fiecare trasatura:
print(min_max_scaler.data_min_ == np.min(X_train, axis=0))
print(min_max_scaler.data_max_ == np.max(X_train, axis=0))

In [ ]:
X_train_scaled = ...
print(np.min(X_train_scaled, axis=0), '\n', np.max(X_train_scaled, axis=0), sep='')

Frecvent se cere atat determinarea parametrilor de transfromare, cat si aplicarea transformarii pe un acelsi set de date:

In [ ]:
X_train_scaled = ...

Transformarea setului de testare se face folosind acelasi obiect de scalare obtinut (fitted) pe setul de antrenare

In [ ]:
X_test_scaled = ...

Daca setul de testare face parte din aceeasi distributie ca si cel de antrenare, ar trebui ca valorile minima si maxime obtinute pe setul de testare sa fie aproximativ 0 si 1:

In [ ]:
#minim pe setul de testare, pe fiecare trasatura
...

In [ ]:
#maxim pe setul de testare, pe fiecare trasatura
...

Exemplul de mai sus se aplica cu mimime modificari altor metode de scalare:

In [ ]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()
X_train_std = standard_scaler.fit_transform(X_train)
print('valori medii: ', np.mean(X_train_std, axis = 0))
print('deviatie standard: ', np.std(X_train_std, axis = 0))

Efectul aplicarii unei astfel de preprocesari este dat mai jos:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#varianta cu date nescalate

knn = KNeighborsClassifier(n_neighbors=3)
...

In [ ]:
knn_scaled = KNeighborsClassifier(n_neighbors=3)
...

Desigur, putem constata si efectul pe datele standardizate:

In [ ]:
knn_std = KNeighborsClassifier(n_neighbors=3)
...

### 7.1.2. Reducerea dimensionalitatii

Frecvent, datele disponibile au un numar mare de dimeniuni. In destule cazuri se poate renunta la unele din ele, fara a peirde foarte multa informatie. In plus, se castiga in viteza de calcul, deoarece se ajunge sa se lucreze cu mai putine trasaturi. In destule situatii se poate ajunge la doua trasaturi numerice care pot fi reprezentate in plan, dand posibilitatea unei explorari initiale. 



Cea mai populara transformare este analiza componentelor principale (Principal Component Analysis, PCA) care se obtine prin metode algebrice relativ simple. 

Bibliografie recomandata pentru prezentare matematica:
1. [A tutorial on Principal Components Analysis](http://www.cs.otago.ac.nz/cosc453/student_tutorials/principal_components.pdf)
1. [A Tutorial on Principal Component Analysis](https://arxiv.org/abs/1404.1100)
1. [PCA Whitening](http://ufldl.stanford.edu/tutorial/unsupervised/PCAWhitening/)

In [ ]:
#sursa: Introduction to Machine Learning with Python, chapter 03

fig, axes = plt.subplots(15, 2, figsize=(10, 20))
malignant = data_cancer.data[data_cancer.target == 0]
benign = data_cancer.data[data_cancer.target == 1]
ax = axes.ravel()
for i in range(30):
    _, bins = np.histogram(data_cancer.data[:, i], bins=50)
    ax[i].hist(malignant[:, i], bins=bins, color='red', alpha=.5)
    ax[i].hist(benign[:, i], bins=bins, color='green', alpha=.5)
    ax[i].set_title(data_cancer.feature_names[i])
    ax[i].set_yticks(())
ax[0].set_xlabel("Feature magnitude")
ax[0].set_ylabel("Frequency")
ax[0].legend(["malignant", "benign"], loc="best")
fig.tight_layout()

In histogramele anterioare se observa ca diferite trasaturi individuale au o putere discriminativa mai mica sau mai mare. Ne intereseaza sa consideram doua tarsaturi (nu neaparat din cele originare, pot fi si combinatii liniare ale acestora) astfel incat separarea intre malign si benign sa fie mai buna.

In [ ]:
X_cancer, y_cancer = data_cancer.data, data_cancer.target

#se aplica in prealabil o scalare a datelor de intrare
...

In [ ]:
#aplicarea PCA
from sklearn.decomposition import PCA
...
# se tiparesc formele matricelor
print(X_scaled.shape)
print(X_pca.shape)

In [ ]:
X_pca_malign = X_pca[y_cancer == 0]
X_pca_benign = X_pca[y_cancer == 1]

plt.figure(figsize=(20, 10))
plt.scatter(X_pca_malign[:, 0], X_pca_malign[:, 1], c='r', marker='^')
plt.scatter(X_pca_benign[:, 0], X_pca_benign[:, 1], c='g', marker='o')
plt.xlabel('PCA feature 1')
plt.ylabel('PCA feature 2')


Trasaturile determinate de PCA sunt obtinute pe baza unor transformari liniare ale trasaturilor din setul originar. Se pot afisa coeficientii transformarii liniare:

In [ ]:
print('Coeficientii (components_) pentru PCA feature 1, respectiv PCA feature 2:', pca.components_)

O alta metoda destul de populara pentru extragerea de trasaturi este t-SNE. O excelenta prezentare a unuia din autorii algoritmului, Laurens van der Maaten, este [aici](https://www.youtube.com/watch?v=RJVL80Gg3lA). Articolele care prezinta variante ale algoritmului sunt [pe siteul autorului](https://lvdmaaten.github.io/tsne/). Exemplificarea se face pe setul de date `digits` din sklearn:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.DESCR

In [ ]:
fig, axes = plt.subplots(2, 5, subplot_kw = {'xticks':(), 'yticks':()})
for ax, img, img_cls in zip(axes.ravel(), digits.images, digits.target_names):
    ax.imshow(img)
    ax.set_title('Cifra ' + str(img_cls))

In [ ]:
#sursa: https://github.com/amueller/introduction_to_ml_with_python/blob/master/03-unsupervised-learning.ipynb

# build a PCA model
pca = PCA(n_components=2)
pca.fit(digits.data)
# transform the digits data onto the first two principal components
digits_pca = pca.transform(digits.data)
colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120", "#535D8E"]
plt.figure(figsize=(10, 10))
plt.xlim(digits_pca[:, 0].min(), digits_pca[:, 0].max())
plt.ylim(digits_pca[:, 1].min(), digits_pca[:, 1].max())
for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_pca[i, 0], digits_pca[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")



Prin t-SNE se obtin trasaturi mult mai bine diferentiate:

In [ ]:
# sursa: https://github.com/amueller/introduction_to_ml_with_python/blob/master/03-unsupervised-learning.ipynb
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
# use fit_transform instead of fit, as TSNE has no transform method
digits_tsne = tsne.fit_transform(digits.data)

In [ ]:
# sursa: https://github.com/amueller/introduction_to_ml_with_python/blob/master/03-unsupervised-learning.ipynb
plt.figure(figsize=(10, 10))
plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)
for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_tsne[i, 0], digits_tsne[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("t-SNE feature 0")
plt.ylabel("t-SNE feature 1")

## 7.2. Clustering-ul
Clusteringul vizeaza obtinerea de partitii ale setului initial de date. Intre elementele care apartin aceluiasi cluster se considera ca exista relatii de similaritate mai mari decat intre elemente care apartin unor clustere diferite. De exemplu, se doreste impartirea unor imagini cu oameni, in grupuri cu similaritate interna; nu se cunoaste nimic despre indentitatea persoanelor din poze sau metadate. 

### 7.2.1 K-means
K-means este cel mai popular algoritm de clustering. El incearca sa grupeze datele in k clustere. Fiecare cluster este definit printr-un centru de greutate (centroid), ale carui coordonate sunt mediile aritmetice ale coordonatelor punctelor care apartin de acelasi cluster. Un punct din setul de instruire sau de testare este asociat cu cel mai apropiat centroid. 

Bibliografie: 
1. [K-means, Stanford CS 221](http://stanford.edu/~cpiech/cs221/handouts/kmeans.html)
1. [K-means and Hierarchical Clustering, Tutorial Slides by Andrew Moore](https://www.autonlab.org/tutorials/kmeans.html)
1. [Curs Sisteme computationale inteligente](https://github.com/lmsasu/cursuri/blob/master/SistemeComputationaleInteligente/SistemeComputationaleInteligente.pdf) sectiunea 8.4

Ideea de baza este de a determina prin pasi succesivi o pozitionare a centroizilor, precum si o impartire a setului initial de instruire in subseturi (posibil, desi arareori, vide) asociate fiecarui centroid. 

![k-means](./images/kmeans.png)
Sursa: ref [1] de mai sus.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

In [ ]:
X, y = make_blobs()
X.shape

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(X[:, 0], X[:, 1], c=y)

#aplicare algoritm kmeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
centroids = ...
plt.scatter(centroids[:, 0], centroids[:, 1], marker='^', c='red')

Urmatoarele situatii sunt defavorabile pentru algoritmul k-means:
1. Numar de clustere necunoscut apriori
1. cazul in care datele nu au forma aproximativ globulara si de diametre egale
1. Densitati de distributie diferite in clustere
1. Alegere neinspirata a pozitiilor centroizilor; pot rezulta centroizi care partitioneaza un grup de puncte; pot rezulta centroizi orfani = fara puncte asociate
1. Nu trateaza bine situatiile in care clusterele nu sunt sferice.

Pentru aceasta ultiam situatie dam exemplul de mai jos:

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

In [ ]:
#reprezentare set de date; culorile sunt optionale
...

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
y_pred = kmeans.predict(X)

#reprezentare clustere
...

Pentru situatia in care determinarea apartenentei de clustere este data de denistatea de repartitie, mai degraba decat de distanta (cum e cazul de mai sus), algoritmi precum DBSCAN sunt recomandati. 

Ideea algoritmului DBSCAN este de a determian clustere ce acopera regiuni dense de date; clusterele sunt separate de regiuni cu densitate mica. DBSCAN nu necesita precizarea apriori a numarului de clustere (cu toate ca are alti hiperparametri ce trebuie specificati), poate eticheta unele date ca fiind zgomot, adica neafiliate niciunui cluster. 

Bibliografie:
1. [A Density-Based Algorithm for Discovering Clusters in Large Spatial Databases with Noise](https://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf)

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN()#valoarea implicita pentru eps e 0.5
...

Rezulattul de mai sus arata ca se obtine un singur cluster. Prin modificarea valorilor hiperparametrilor eps si min_samples se obtin rezultate complet diferite:

In [ ]:
dbscan = ...

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter...